In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
#indices of the words represent the values in the lookup table
word_to_idx = {"I": 0, "love": 1, "eating":2, "and":3, "sleeping":4}
embeddings = nn.Embedding(5, 7)    # 5 words in vocab, embedding size is 7
word_index = torch.tensor([word_to_idx["love"]])
print(word_index)

tensor([1])


In [4]:
word_index = torch.tensor([word_to_idx["eating"]])
print(word_index)

tensor([2])


In [5]:
love = embeddings(word_index)
print(love)
print(love.shape)

tensor([[-0.8004, -1.8475,  0.0467,  0.7019,  1.0013, -1.5077,  0.3294]],
       grad_fn=<EmbeddingBackward>)
torch.Size([1, 7])


In [6]:

all_ind = torch.tensor([w for w in range(5)], dtype = torch.long)
all_words = embeddings(all_ind)
print(all_ind)
print(all_words)
print(all_words.shape)

tensor([0, 1, 2, 3, 4])
tensor([[-0.3189, -0.4391, -0.5386, -1.3157,  2.2331,  1.1157,  1.7661],
        [ 0.5769, -0.5049,  0.1762,  0.9277,  0.2498, -0.4185, -0.0343],
        [-0.8004, -1.8475,  0.0467,  0.7019,  1.0013, -1.5077,  0.3294],
        [-1.7943,  1.8479, -0.9883,  0.5175,  1.5312, -0.1653,  0.3402],
        [ 0.8644,  0.1463,  0.0526,  0.5615, -1.4194,  0.1949, -1.2298]],
       grad_fn=<EmbeddingBackward>)
torch.Size([5, 7])


In [8]:
#skip gram model
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])
print('=========================')
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]
[519.8486161231995, 517.4153919219971, 514.9989900588989, 512.5988526344299, 510.2130618095398, 507.8423047065735, 505.48550391197205, 503.14706683158875, 500.82103633880615, 498.5063991546631]
